In [1]:
import glob
from music21 import converter, instrument, note, chord
br = 0
notes = []
for file in glob.glob("mozart/*.mid"):
    print(file)
    midi = converter.parse(file)
    notes_to_parse = None
    parts = instrument.partitionByInstrument(midi)
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            for nt in element:
                notes.append(str(nt.pitch))
        notes.append('420')
    br += 1
    if br > 4:
        break

print(notes)

music21: Certain music21 functions might need the optional package matplotlib;
                  if you run into errors, install it by following the instructions at
                  http://mit.edu/music21/doc/installing/installAdditional.html


mozart\sonata01-1.mid
mozart\sonata01-2.mid
mozart\sonata01-3.mid
mozart\sonata02-1.mid
mozart\sonata02-2.mid


['C5', 'G4', 'E4', '420', 'C3', '420', 'C4', '420', 'B3', '420', 'C4', '420', 'C3', '420', 'C5', '420', 'B4', '420', 'D5', '420', 'C5', '420', 'E5', '420', 'D5', '420', 'F5', '420', 'E5', '420', 'G5', '420', 'F5', '420', 'A5', 'F5', 'D5', 'A4', '420', 'F3', 'F2', '420', 'G5', '420', 'F5', '420', 'E5', '420', 'D5', '420', 'C5', '420', 'B4', '420', 'C5', '420', 'E3', '420', 'E5', '420', 'D5', '420', 'F3', '420', 'F5', '420', 'B4', '420', 'G3', '420', 'D5', '420', 'C5', 'G4', 'E4', '420', 'C3', '420', 'C4', '420', 'B3', '420', 'C4', '420', 'C3', '420', 'C5', '420', 'B4', '420', 'D5', '420', 'C5', '420', 'E5', '420', 'D5', '420', 'F5', '420', 'E5', '420', 'G5', '420', 'F5', '420', 'A5', 'F5', 'D5', 'A4', '420', 'F3', 'F2', '420', 'G5', '420', 'F5', '420', 'E5', '420', 'D5', '420', 'C5', '420', 'B4', '420', 'C5', '420', 'E3', '420', 'E5', '420', 'D5', '420', 'F3', '420', 'F5', '420', 'B4', '420', 'G3', '420', 'D5', '420', 'C5', '420', 'C4', '420', 'G4', '420', 'C5', '420', 'D5', '420', 'E4'

In [11]:
import numpy
from keras.utils import np_utils
sequence_length = 20
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
n_vocab = len(pitchnames)
print(pitchnames)
print(n_vocab)
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = np_utils.to_categorical(network_output)

print(network_input.shape[1], network_input.shape[2])

['420', 'A1', 'A2', 'A3', 'A4', 'A5', 'B-1', 'B-2', 'B-3', 'B-4', 'B-5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C#2', 'C#3', 'C#4', 'C#5', 'C#6', 'C2', 'C3', 'C4', 'C5', 'C6', 'D2', 'D3', 'D4', 'D5', 'D6', 'E-2', 'E-3', 'E-4', 'E-5', 'E-6', 'E2', 'E3', 'E4', 'E5', 'E6', 'F#2', 'F#3', 'F#4', 'F#5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G#1', 'G#2', 'G#3', 'G#4', 'G#5', 'G1', 'G2', 'G3', 'G4', 'G5']
60
20 1


In [137]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Activation
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [138]:
from keras.callbacks import ModelCheckpoint

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=False,  
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=200, batch_size=128, callbacks=callbacks_list)

Epoch 1/200
24615/24615 [==============================] - 134s 5ms/step - loss: 2.6264
Epoch 2/200
24615/24615 [==============================] - 122s 5ms/step - loss: 2.2233
Epoch 3/200
24615/24615 [==============================] - 122s 5ms/step - loss: 2.1612
Epoch 4/200
24615/24615 [==============================] - 122s 5ms/step - loss: 2.1140
Epoch 5/200
24615/24615 [==============================] - 122s 5ms/step - loss: 2.0627
Epoch 6/200
24615/24615 [==============================] - 122s 5ms/step - loss: 2.0163
Epoch 7/200
24615/24615 [==============================] - 122s 5ms/step - loss: 1.9627
Epoch 8/200
24615/24615 [==============================] - 122s 5ms/step - loss: 1.8987
Epoch 9/200
24615/24615 [==============================] - 122s 5ms/step - loss: 1.8279
Epoch 10/200
24615/24615 [==============================] - 122s 5ms/step - loss: 1.7262
Epoch 11/200
24615/24615 [==============================] - 122s 5ms/step - loss: 1.6199
Epoch 12/200
24615/24615 [====

24615/24615 [==============================] - 121s 5ms/step - loss: 0.0521
Epoch 185/200
24615/24615 [==============================] - 121s 5ms/step - loss: 0.0484
Epoch 186/200
24615/24615 [==============================] - 121s 5ms/step - loss: 0.0514
Epoch 187/200
24615/24615 [==============================] - 122s 5ms/step - loss: 0.0528
Epoch 188/200
24615/24615 [==============================] - 122s 5ms/step - loss: 0.0511
Epoch 189/200
24615/24615 [==============================] - 121s 5ms/step - loss: 0.0537
Epoch 190/200
24615/24615 [==============================] - 121s 5ms/step - loss: 0.0488
Epoch 191/200
24615/24615 [==============================] - 121s 5ms/step - loss: 0.0507
Epoch 192/200
24615/24615 [==============================] - 121s 5ms/step - loss: 0.0499
Epoch 193/200
24615/24615 [==============================] - 121s 5ms/step - loss: 0.0535
Epoch 194/200
24615/24615 [==============================] - 121s 5ms/step - loss: 0.0511
Epoch 195/200
24615/2461

In [17]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import Activation
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
# Load the weights to each node
model.load_weights('mozart_weights/weights-improvement-30-0.2012-bigger.hdf5')


In [18]:
start = numpy.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = [numpy.random.randint(0, n_vocab-1) for x in range(sequence_length)]
prediction_output = []
# generate 500 notes
for note_index in range(500 + sequence_length):
    prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input / float(n_vocab)
    prediction = model.predict(prediction_input, verbose=0)
    print(prediction)
    index = numpy.argmax(prediction)
    print(prediction[0][index])
    result = int_to_note[index]
    prediction_output.append(result)
    pattern = numpy.append(pattern, numpy.array([index]))
    pattern = pattern[1:len(pattern)]
prediction_output = prediction_output[sequence_length:]

[[4.32897121e-01 4.55802118e-10 2.52835964e-11 1.27073978e-08
  8.54044786e-12 2.39128051e-10 1.12351772e-08 7.89945247e-08
  5.66984892e-01 9.34825620e-11 2.06357917e-11 1.86542137e-09
  3.62469912e-08 1.14564231e-04 2.15316751e-10 2.17556279e-10
  1.35427012e-08 4.54249660e-09 9.09204823e-10 3.11267993e-13
  2.47603518e-11 1.66155867e-10 1.89143670e-08 1.38207029e-10
  1.45250548e-10 2.78166379e-10 2.49431392e-10 8.26023069e-11
  4.23145177e-15 9.23584609e-17 3.03619197e-14 1.42003387e-09
  1.87977411e-11 6.88905460e-11 1.38118935e-11 3.30113464e-10
  1.33099443e-09 1.91119162e-10 6.68642279e-13 8.58344645e-12
  1.22654612e-07 3.99669187e-09 2.02317091e-12 6.15443738e-11
  1.90445142e-11 1.93242062e-08 4.49730982e-11 3.31184177e-11
  2.62885332e-14 9.74175740e-10 6.43208686e-09 1.52400901e-07
  5.83514122e-07 5.65032074e-08 2.39618657e-06 3.56470353e-10
  3.65436614e-09 2.42841658e-08 4.65022292e-11 4.39118787e-14]]
0.5669849
[[9.58438873e-01 1.01497722e-06 2.98986590e-04 1.93339531e

1.0
[[1.93656410e-06 1.31399496e-08 2.47557466e-07 2.30592057e-09
  4.93215964e-07 4.69592545e-04 7.12792447e-09 4.64097843e-10
  5.63853894e-11 1.69279811e-05 3.18213123e-09 2.40998221e-10
  6.97716052e-10 3.67174920e-07 2.77239309e-07 2.46287044e-07
  3.87069987e-10 2.46957343e-09 1.72864323e-09 2.55418104e-08
  5.32568781e-11 3.18230109e-09 1.10744968e-05 2.73204828e-03
  8.08093138e-03 1.02386803e-05 1.09961440e-09 1.08983784e-06
  1.03605316e-06 3.67378034e-02 9.04621658e-08 5.91074190e-10
  2.59435762e-09 6.63577138e-10 4.42214173e-07 1.39748368e-09
  2.36376803e-08 1.45127806e-05 1.24798942e-04 3.85768302e-02
  3.04263559e-10 5.18849008e-09 2.58179593e-08 2.19404717e-09
  5.09565798e-05 8.99583630e-10 3.93829103e-09 4.91917298e-08
  3.81838290e-05 7.95587242e-01 1.74388450e-08 1.48337289e-08
  6.08675155e-09 6.79570675e-08 7.52370788e-09 1.10930043e-09
  2.45579179e-09 3.75944920e-09 8.64222841e-07 1.17541485e-01]]
0.79558724
[[9.70524251e-01 7.54433415e-07 4.30118829e-10 1.3666

0.5994421
[[9.99999642e-01 7.61949011e-12 3.26666775e-12 1.26366495e-11
  1.97217993e-08 1.95237964e-13 7.10006012e-12 2.90615781e-10
  2.42209031e-13 4.50454170e-12 2.16603211e-12 1.14515966e-10
  7.71336228e-13 7.25421581e-14 1.45274293e-10 2.76489525e-13
  4.78642716e-11 1.34552864e-11 1.77180666e-12 1.77147141e-09
  6.95721963e-13 3.65922006e-12 1.37745921e-13 9.17042846e-15
  3.15612825e-13 1.32385368e-13 8.75184907e-12 6.82342016e-10
  4.08530440e-07 7.70931852e-09 2.00788188e-11 3.19869619e-13
  1.85645756e-13 1.11181995e-12 3.31120634e-12 1.35107836e-11
  1.70840677e-13 3.88095806e-16 2.94251665e-15 1.18488357e-12
  7.94425777e-13 7.78687705e-12 1.92166560e-11 3.53081342e-10
  1.06628761e-09 3.96769943e-12 3.21047008e-12 4.44708853e-12
  2.61466671e-09 1.19466159e-09 2.94912879e-12 1.57475378e-12
  3.47160106e-10 1.24630209e-10 1.72106388e-13 2.42302944e-12
  1.23306559e-10 8.08119544e-12 1.24690180e-09 2.11223945e-08]]
0.99999964
[[1.09569213e-04 1.50527612e-07 3.63090642e-08 

[[1.6065077e-04 1.8162125e-07 1.1675199e-05 3.2093056e-04 3.0857660e-03
  5.1082054e-05 6.5097261e-08 2.9246718e-07 3.1843518e-03 5.5136011e-06
  4.6628167e-05 4.9932203e-07 1.4663265e-05 1.1165958e-04 1.3208115e-03
  1.3729027e-03 1.2757513e-07 4.6081568e-06 1.3934613e-04 1.8824863e-05
  9.5674864e-07 2.6120572e-06 1.3778502e-05 1.5876249e-04 3.6162667e-03
  1.2228686e-03 1.0361296e-05 5.9697719e-04 3.1566303e-02 3.2075111e-02
  1.7435166e-06 2.4913146e-07 5.3157551e-06 4.2562756e-06 2.1522927e-04
  1.8591446e-07 1.1871894e-06 1.7728827e-04 6.6001244e-06 4.6023091e-05
  2.1929204e-06 4.3675507e-07 1.3225349e-06 1.1195275e-03 7.3373999e-06
  1.8718863e-07 2.2257287e-07 9.0309077e-06 1.3313127e-05 1.0208639e-05
  4.5810030e-08 5.0474654e-07 1.6508520e-05 5.0362960e-05 6.1574515e-06
  1.7842119e-07 9.8474506e-05 1.1108111e-02 8.9417851e-01 1.3804716e-02]]
0.8941785
[[9.9999559e-01 1.2159264e-08 2.4549388e-10 5.6855069e-11 1.4769828e-10
  3.2179657e-09 7.9295837e-10 4.0531550e-11 5.240135

  1.16544605e-08 4.40142749e-08 1.98270897e-07 2.87724133e-09]]
0.9999975
[[2.33534720e-05 2.59576227e-05 2.08423330e-06 2.86733247e-02
  5.64789653e-01 4.90449485e-04 3.05010508e-05 3.74131778e-05
  1.88013164e-05 1.31099736e-02 5.31008081e-05 9.26142638e-06
  1.05485142e-05 3.75624921e-04 1.03593757e-03 1.38016221e-08
  3.44735381e-05 1.23969758e-05 5.25845680e-05 1.22865677e-04
  4.49135841e-05 3.53348298e-07 3.03673062e-07 3.18490173e-04
  2.56167026e-03 1.60910677e-05 1.18484140e-05 2.45069852e-04
  7.18520069e-03 1.80394724e-02 2.41703256e-05 1.15093908e-05
  2.43764644e-05 2.58261589e-05 3.76953576e-05 1.60754844e-05
  1.41881346e-05 3.36141566e-05 1.58918109e-02 3.38667512e-01
  3.34889955e-06 1.82917047e-05 1.14008517e-03 1.68674940e-03
  4.91988176e-05 1.83218053e-05 5.84600726e-04 4.99377609e-04
  6.54073607e-04 2.16729613e-03 1.79173949e-05 9.83642258e-06
  8.06155585e-05 3.38001497e-04 2.78156153e-06 9.71602276e-06
  1.05817435e-05 1.48804800e-04 2.77394411e-05 4.54190973e

...

0.9863988
[[5.1181496e-04 2.0574307e-04 3.6043028e-05 1.2240075e-05 3.4725736e-03
  7.3851678e-03 3.7096444e-04 5.9381343e-04 8.9089939e-05 5.9951213e-05
  7.9130626e-04 1.1904628e-03 1.5779233e-02 7.2684183e-05 3.0749468e-06
  3.4382464e-03 4.7527664e-04 9.0688378e-02 3.2220221e-06 3.0295343e-05
  2.7306340e-04 2.2987207e-02 6.7624234e-02 1.4028713e-03 2.2423335e-05
  2.0695601e-02 4.5041116e-03 5.3632739e-03 1.2054954e-05 8.6382286e-05
  2.7875756e-03 3.1699828e-04 4.9616061e-02 3.8185070e-04 5.6408942e-01
  2.0310121e-05 2.7864263e-04 5.0621047e-03 3.2239015e-05 1.0612053e-03
  2.5389480e-04 2.5882848e-04 6.8990856e-05 2.9159570e-05 2.5519342e-03
  4.2042613e-04 1.4601514e-03 8.9089602e-04 2.0628278e-03 3.8796980e-02
  2.5679517e-05 1.2252135e-04 2.8041176e-02 1.6115222e-02 3.0729346e-04
  6.5254666e-05 1.3152382e-04 4.6742000e-05 1.6091908e-04 3.6362391e-02]]
0.5640894
[[9.46839392e-01 9.03191264e-08 1.55145563e-09 9.44890743e-09
  1.80842599e-05 5.23904338e-04 4.45535484e-08 7.471

  3.63331964e-07 3.96529600e-07 4.58442573e-09 5.16202081e-10]]
0.99998534
[[2.12160612e-05 4.30793079e-05 2.66851330e-05 9.67492466e-04
  1.13001084e-02 1.34752430e-02 2.33905175e-05 7.80418268e-05
  1.14022219e-03 2.95961071e-02 3.71493516e-04 1.38044215e-05
  4.98306763e-05 7.77084427e-03 4.32250872e-02 4.46758961e-04
  1.36506487e-05 9.64631727e-06 6.29155966e-06 4.73162567e-04
  1.02441503e-04 4.11998742e-04 9.64027643e-03 1.20750204e-01
  2.37391628e-02 3.21173255e-04 4.76039313e-05 9.65537853e-04
  7.35838898e-03 4.02849950e-02 4.35894435e-05 1.51740933e-05
  4.04565326e-05 7.18209340e-05 5.64216811e-04 1.64719204e-05
  3.78768709e-05 1.08970649e-04 5.33549301e-03 6.33056462e-01
  3.62829887e-05 1.15512648e-05 1.84939586e-06 6.06352223e-05
  1.84724977e-05 8.87712577e-06 1.72895761e-05 2.73641308e-05
  4.45308257e-03 2.37388182e-02 1.26282430e-05 1.51811573e-05
  4.97658737e-04 1.80047727e-03 1.38362916e-03 1.59941319e-05
  5.99647028e-05 3.16185004e-04 5.04347787e-04 1.50552988

[[2.5528620e-03 1.2610663e-05 3.2868360e-05 7.2118704e-04 2.0626327e-03
  8.1846537e-03 5.5593569e-06 2.8401974e-04 2.3381930e-04 1.0738802e-01
  4.3406809e-04 1.0858172e-05 1.8246580e-03 3.5018578e-02 6.3388702e-04
  1.2845009e-05 4.3825312e-06 5.6219928e-04 3.1916058e-04 1.0070940e-02
  1.7261812e-04 7.1768576e-05 1.7571477e-03 1.0308871e-01 3.0529089e-03
  1.2455041e-04 7.6573924e-05 2.1643235e-04 1.1695146e-03 3.7694403e-03
  3.9478613e-04 9.9531208e-05 1.3971425e-03 3.5073277e-05 1.5252663e-03
  1.9123397e-06 8.8640285e-05 1.6827413e-03 5.3217232e-01 1.2263144e-01
  1.4603536e-05 8.0747432e-06 4.7353828e-05 3.9383830e-04 2.7574003e-03
  1.0968200e-05 2.1161619e-04 4.7532590e-03 2.4462689e-03 1.5641734e-02
  2.8355396e-06 5.9593873e-05 1.1833654e-04 1.9539837e-02 4.8784530e-04
  7.8206722e-07 1.3201243e-06 5.2277400e-04 6.3282545e-03 2.7551081e-03]]
0.5321723
[[9.9999976e-01 1.4508249e-09 1.5684060e-11 2.7016908e-10 2.3942081e-11
  2.5007417e-08 1.8950920e-10 1.5506733e-09 2.732211

0.6075224
[[9.9755967e-01 4.1915073e-06 1.6638411e-07 1.4858299e-05 1.0637571e-03
  2.8624546e-04 1.9022015e-06 2.3794634e-07 4.2324103e-08 2.6993595e-07
  2.3894825e-07 6.3698157e-05 9.5217893e-06 7.5444186e-06 1.5207312e-04
  3.9432369e-05 2.6151631e-06 8.0633472e-06 7.4322679e-07 7.8277872e-06
  3.0322913e-06 6.9419671e-05 1.6124125e-06 3.2231721e-08 2.9925471e-07
  8.4178118e-06 2.4247911e-05 1.2953096e-06 8.1212811e-08 5.6910362e-06
  2.1289693e-06 5.5409294e-07 3.5515725e-07 3.7153754e-06 1.8676168e-06
  8.3183329e-07 1.8299486e-06 1.7848052e-08 1.1405858e-08 5.2261572e-07
  2.9922119e-06 8.4495775e-07 1.8538747e-06 1.5230445e-06 1.2813047e-06
  8.7705905e-07 2.7125686e-06 4.0189439e-05 1.8996634e-06 8.5870028e-07
  5.6105233e-07 4.3121926e-08 1.1454442e-06 3.8499142e-08 4.5155744e-07
  4.8691078e-07 8.2278217e-07 4.8480008e-04 9.9332843e-05 8.1966709e-06]]
0.99755967
[[2.57948268e-04 1.62468859e-06 2.36594701e-06 1.28457346e-03
  1.43947555e-02 2.03710893e-04 6.98605959e-07 1.62

  5.06581599e-03 4.11313307e-03 9.07697063e-03 5.83176334e-05]]
0.7444897
[[9.9999356e-01 3.1637833e-12 1.4510947e-10 8.5180951e-10 9.0013847e-13
  3.9583820e-10 1.2531056e-12 9.6747063e-12 4.8193166e-13 2.9713072e-11
  2.7252000e-15 1.2285443e-13 1.2122989e-13 1.3506064e-11 2.2622488e-12
  8.7218081e-14 4.8874853e-13 1.0132780e-11 2.9162780e-11 2.8005144e-12
  7.5704437e-14 1.9301511e-14 9.0109378e-09 2.3627431e-06 3.0882202e-08
  1.1196590e-09 3.6230567e-12 1.2748274e-07 3.7640616e-06 6.1816174e-08
  4.5485716e-12 8.6163166e-14 4.2929893e-14 2.2776437e-13 4.6557921e-11
  3.2577066e-12 4.7958654e-11 7.6070721e-12 1.8178419e-11 2.0664849e-10
  2.7449128e-12 1.3677524e-11 8.9489215e-12 1.7566905e-12 4.4030135e-12
  2.6171081e-11 5.2853955e-10 1.7057195e-10 2.0944304e-08 1.0992428e-10
  2.3009530e-12 1.3250418e-11 7.0951670e-13 7.5200384e-12 5.7451210e-12
  4.1292724e-12 4.8179429e-11 4.1702300e-13 2.4079742e-14 2.7115536e-11]]
0.99999356
[[9.50526896e-07 3.90617388e-05 2.46920553e-03 5.

In [21]:
offset = 0
output_notes = []
cur_notes = []
for pattern in prediction_output:
    if pattern == '420':
        notes = []
        for cur_note in cur_notes:
            new_note = note.Note(cur_note)
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        if len(cur_notes) > 0:
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        offset += 0.6
        cur_notes.clear()
    else:
        cur_notes.append(pattern)
        
print(output_notes)
        

[<music21.chord.Chord A4>, <music21.chord.Chord F3>, <music21.chord.Chord G4>, <music21.chord.Chord F#4>, <music21.chord.Chord G4>, <music21.chord.Chord D5>, <music21.chord.Chord D5>, <music21.chord.Chord G4>, <music21.chord.Chord C5>, <music21.chord.Chord A4>, <music21.chord.Chord F#4>, <music21.chord.Chord C5>, <music21.chord.Chord C#5>, <music21.chord.Chord F#4>, <music21.chord.Chord C#6>, <music21.chord.Chord A3>, <music21.chord.Chord E-4>, <music21.chord.Chord F#4>, <music21.chord.Chord E-5>, <music21.chord.Chord A3>, <music21.chord.Chord C4>, <music21.chord.Chord F#4>, <music21.chord.Chord A3>, <music21.chord.Chord C4>, <music21.chord.Chord D5>, <music21.chord.Chord B-3>, <music21.chord.Chord C#4>, <music21.chord.Chord G4>, <music21.chord.Chord A4>, <music21.chord.Chord F4>, <music21.chord.Chord D4>, <music21.chord.Chord A4>, <music21.chord.Chord F5>, <music21.chord.Chord G4>, <music21.chord.Chord B-4>, <music21.chord.Chord G3>, <music21.chord.Chord A4>, <music21.chord.Chord A5>,

In [132]:
offset = 0
tempo = 0.3
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += tempo

In [22]:
from music21 import stream
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='test_output_5.mid')

'test_output_5.mid'

In [72]:
print(output_notes)

[<music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.note.Note A>, <music21.n